## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import time

%matplotlib qt

def get_calibration(img_list):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    for fname in img_list:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return (objpoints, imgpoints)

#         # Draw and display the corners
#         img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#         cv2.imshow('img',imgx)
#         cv2.waitKey(500)

# Make a list of calibration images
calibration_images = glob.glob('camera_cal/calibration*.jpg')

# global calibration values
objpoints, imgpoints = get_calibration(calibration_images)
target_dimensions = (720, 1280)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, target_dimensions, None, None)

# nx, ny values to use for undistorting
max_points = np.max(objpoints[0], axis=0) + 1
nx = int(max_points[0])
ny = int(max_points[1])



## Now, let's load up a test image and do some undistortion magic

In [2]:
# this method assumes both `orig` and `result` were loaded with cv2.imread which reads in BGR format
# thus we convert to RGB before showing
def plot_images(orig, result):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    if len(orig) != 3:
        orig = make_3_channel_image_from_binary_channel(orig)
    if len(result) != 3:
        result = make_3_channel_image_from_binary_channel(result)
    ax1.imshow(orig)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(result)
    ax2.set_title('Processed Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    
def plot_image(img):
    """
    Creates a plot and displays an image on it
    """
    figure = plt.figure(figsize = (18,10))
    axes = figure.add_subplot(111)

    is_grayscale = len(img) != 3
    if is_grayscale is True:
        axes.imshow(img, cmap='gray', aspect='auto', interpolation='nearest') 
    else:
        axes.imshow(img, aspect='auto', interpolation='nearest') 

def weighted_img(img, initial_img, α=1, β=0.6, γ=0.):
    """
    Overlays a couple images together
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

def make_3_channel_image_from_binary_channel(bin_channel):
    """
    Takes a single-channel binary image (1s and 0s) and triples the channels
    """
    return np.dstack((bin_channel, bin_channel, bin_channel)) * 255
    
def sobel(img, sobel_kernel, mag_thresh=(0, 255), dir_thresh=(0, np.pi/2)):
    mag_thresh_min, mag_thresh_max = mag_thresh
    dir_thresh_min, dir_thresh_max = dir_thresh
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)

    # directional sobel
    sobel_dir = np.arctan2(abs_sobel_y, abs_sobel_x)
    dir_binary_output = np.zeros_like(sobel_dir)
    dir_binary_output[(sobel_dir >= dir_thresh_min) & (sobel_dir <= dir_thresh_max)] = 1
    
    # magnitude sobel
    abs_sobel_xy = np.sqrt(abs_sobel_x ** 2 + abs_sobel_y ** 2)
    scaled_sobel = np.uint8(255 * abs_sobel_xy / np.max(abs_sobel_xy))
    mag_binary_output = np.zeros_like(scaled_sobel)
    mag_binary_output[(scaled_sobel >= mag_thresh_min) & (scaled_sobel <= mag_thresh_max)] = 1

    return np.uint8(dir_binary_output), mag_binary_output


In [3]:
def draw_lane_on_image(new_image, left_line, right_line):
    left_x, left_y = left_line.get_x_and_y()
    right_x, right_y = right_line.get_x_and_y()

    pts_left = np.array([np.transpose(np.vstack([left_x, left_y]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_x, right_y])))])
    
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(new_image, np.int_([pts]), (255, 255, 0))
    cv2.polylines(new_image, np.int32([pts_left]), 0, [0,255,0], 15)
    cv2.polylines(new_image, np.int32([pts_right]), 0, [0,255,0], 15)
    
    # todo add deviation too
    
# def undistort_image(img, objpoints, imgpoints):
#     return cv2.undistort(img, mtx, dist, None)

def find_and_update_lane_lines(binary_warped, left_line, right_line, visualize=False):
    if left_line.detected == True and right_line.detected == True:
        leftx, lefty, rightx, righty = find_lane_from_prior(
            binary_warped,
            left_line.average_fit,
            right_line.average_fit,
            visualize
        )
    else:
        leftx, lefty, rightx, righty = find_lane_sliding_windows(binary_warped, visualize)

    left_line.update_line(lefty, leftx)
    right_line.update_line(righty, rightx)

def get_pixels_in_window(arr, bounds):
    nonzeroy, nonzerox = arr
    min_x, max_x, min_y, max_y = bounds
    return ((nonzeroy >= min_y) & (nonzeroy < max_y) & (nonzerox >= min_x) &  (nonzerox < max_x)).nonzero()[0]

def get_x_from_poly(fit_fn, y):
    a, b, c = fit_fn
    return a * y ** 2 + b * y + c

def get_pixels_from_prior(nonzero, fit_fn, margin, height):
    result = []
    nonzeroy, nonzerox = nonzero
    for i in range(height):
        x = get_x_from_poly(fit_fn, i)
        min_x = x - margin
        max_x = x + margin
        ((i == nonzeroy) & (nonzerox >= min_x) & (nonzerox < max_x)).nonzero()
        result.extend(((i == nonzeroy) & (nonzerox >= min_x) & (nonzerox < max_x)).nonzero()[0])
    return result

In [4]:
def get_image_of_found_lane(warped, max_y, visualize=False):    
    find_and_update_lane_lines(warped, left_line, right_line, visualize)

    warped_three_channel = make_3_channel_image_from_binary_channel(warped)    
    new_image = np.zeros_like(warped_three_channel)

    draw_lane_on_image(new_image, left_line, right_line)
    return new_image

def find_lane_from_prior(binary_warped, left_fit, right_fit, visualize=False):
    margin = 50

    # get the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy, nonzerox = nonzero
    
    left_lane_inds = get_pixels_from_prior(nonzero, left_fit, margin, binary_warped.shape[0])
    right_lane_inds = get_pixels_from_prior(nonzero, right_fit, margin, binary_warped.shape[0])
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def find_lane_sliding_windows(binary_warped, visualize=False):
    img_height = binary_warped.shape[0]

    num_windows = 7
    margin = 150
    minpix = 100

    # get the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy, nonzerox = nonzero

    histogram = np.sum(binary_warped[img_height // 2:,:], axis=0)
    out_img = make_3_channel_image_from_binary_channel(binary_warped)

    # find midpoint of histogram
    midpoint = np.int(histogram.shape[0] // 2)
    # take midpoint of left and right sides as the start location for the search
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    window_height = np.int(img_height // num_windows)

    # some initial values for the search
    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(num_windows):
        win_y_low = img_height - (window + 1) * window_height
        win_y_high = img_height - window * window_height

        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        if visualize is True:
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(255,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(255,255,0), 2) 
        
        good_left_inds = get_pixels_in_window(nonzero, (win_xleft_low, win_xleft_high, win_y_low, win_y_high))
        good_right_inds = get_pixels_in_window(nonzero, (win_xright_low, win_xright_high, win_y_low, win_y_high))

        if len(good_left_inds) > 0:
            left_lane_inds.append(good_left_inds)
        if len(good_right_inds) > 0:
            right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    if len(left_lane_inds) > 0:
        left_lane_inds = np.concatenate(left_lane_inds)
    if len(right_lane_inds) > 0:
        right_lane_inds = np.concatenate(right_lane_inds)
    if visualize is True:
        plot_image(out_img)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty


In [7]:
def find_edges(img, debug=False):
    img = np.copy(img)

#     dir_thresh = (0.7, 1.1)
    dir_thresh = (0.7, 1.2)
    mag_thresh = (70, 150)

    yellow_h_thresh = (15, 33)
    yellow_s_thresh = (50, 255)
    yellow_v_thresh = (200, 255)
    white_h_thresh = (0, 179)
    white_s_thresh = (0, 45)
    white_v_thresh = (220, 255)

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    v = hsv[:,:,2]

    # threshold color channels to capture yellow
    h_binary = np.zeros_like(h)
    s_binary = np.zeros_like(s)
    v_binary = np.zeros_like(v)
    h_binary[(h >= yellow_h_thresh[0]) & (h <= yellow_h_thresh[1])] = 1
    s_binary[(s >= yellow_s_thresh[0]) & (s <= yellow_s_thresh[1])] = 1
    v_binary[(v >= yellow_v_thresh[0]) & (v <= yellow_v_thresh[1])] = 1
    yellow_binary = h_binary & s_binary & v_binary
    
    # threshold color channels to capture white
    h_binary = np.zeros_like(h)
    s_binary = np.zeros_like(s)
    v_binary = np.zeros_like(v)
    h_binary[(h >= white_h_thresh[0]) & (h <= white_h_thresh[1])] = 1
    s_binary[(s >= white_s_thresh[0]) & (s <= white_s_thresh[1])] = 1
    v_binary[(v >= white_v_thresh[0]) & (v <= white_v_thresh[1])] = 1
    white_binary = h_binary & s_binary & v_binary

    dir_binary_output, mag_binary_output = sobel(img, sobel_kernel=5, mag_thresh=mag_thresh, dir_thresh=dir_thresh)

    result = white_binary | yellow_binary | (dir_binary_output & mag_binary_output)

    if debug is True:
        plot_image(yellow_binary)
        plot_image(white_binary)
        plot_image(dir_binary_output)
        plot_image(mag_binary_output)
        plot_image(result)
    # binary `or` here makes sure we capture both magnitude and directional sobels (?)
    # todo: fix above line
    return result
    

### # Define a class to receive the characteristics of each line detection
class Line():
    # class var to tune averaging
    num_frames_to_average = 4

    def __init__(self, max_x, max_y, ym_per_pix, xm_per_pix):
        self.max_x = max_x
        self.max_y = max_y
        self.best_y = np.linspace(0, self.max_y - 1, self.max_y)
        self.best_x = []
        self.ym_per_pix = ym_per_pix
        self.xm_per_pix = xm_per_pix

        # was the line detected in the last iteration?
        self.detected = False

        # x values of the last n fits of the line
        self.recent_xfitted = [] 

        #average x values of the fitted line over the last n iterations
        self.all_fits = []  
        self.all_fits_world = []
        self.average_fit = []
        self.average_fit_world = []

        # radius of curvature of the line in some units
        self.average_curvature = None 
        self.all_curvature_values = []

        # distance in meters of vehicle center from the line
        self.offset_world = 0

    def get_curvature(self, average_fit, y, ym_per_pix):
        a, b, c = average_fit
        return ((1 + (2 * a * y * ym_per_pix + b) ** 2) ** 1.5) / abs(2 * a)
        
    def update_line(self, laney, lanex):
        
        coefficients = np.polyfit(laney, lanex, 2) if len(lanex) > 0 and len(laney) > 0 else None
        coefficients_world = np.polyfit(laney * self.ym_per_pix, lanex * self.xm_per_pix, 2) if len(lanex) > 0 and len(laney) > 0 else None
       
        if len(self.average_fit) == 0:
            self.average_fit = coefficients
            self.average_fit_world = coefficients_world

        self.detected = True
        self.all_fits.append(coefficients)
        self.all_fits_world.append(coefficients_world)

        self.all_curvature_values.append(self.get_curvature(self.average_fit_world, self.max_y - 1, self.ym_per_pix))

        if len(self.all_fits) > self.num_frames_to_average:
            self.all_fits.pop(0)
            self.all_fits_world.pop(0)
            self.all_curvature_values.pop(0)

        self.average_fit = np.sum(self.all_fits, axis=0) / len(self.all_fits)
        self.average_fit_world = np.sum(self.all_fits_world, axis=0) / len(self.all_fits_world)
        self.average_curvature = sum(self.all_curvature_values) / len(self.all_curvature_values)

        self.best_x = get_x_from_poly(self.average_fit, self.best_y)

    def get_x_and_y(self):
        return self.best_x, self.best_y

img_height, img_width = target_dimensions
lane_width = 3.66
# placeholder values
# meters per pixel in y dimension
# estimated by using average length of and distance between white lane dashes (total of ~125 feet covered)
ym_per_pix = 38.1 / 720

# meters per pixel in the x direction;
# estimated from average lane width (~12 feet) and the number of pixels
# between the lane lines at the bottom of the screen (assume y=720 is directly below the camera)
xm_per_pix = lane_width / 620

# instantiate a couple lines
left_line = Line(img_width, img_height, ym_per_pix, xm_per_pix)
right_line = Line(img_width, img_height, ym_per_pix, xm_per_pix)

# top of the trapezoid
img_transform_min_y = 450
# offset from the center of the image
img_transform_top_center_offset = 49
# offset from the side of the image
img_transform_bottom_side_offset = 150

src = np.float32([
    [img_transform_bottom_side_offset, img_height],
    [img_width / 2 - img_transform_top_center_offset, img_transform_min_y],
    [img_width / 2 + img_transform_top_center_offset, img_transform_min_y],
    [img_width - img_transform_bottom_side_offset, img_height]
])
dst = np.float32([
    [300, img_height],
    [300, 0],
    [img_width - 300, 0],
    [img_width - 300, img_height]
])

M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)
debug = False

# a somewhat straight line is something with a radius curvature radius of greater than 10km
# (seems reasonable to me)
somewhat_straight_curvature_radius = 5000

def format_in_meters(some_num):
    rounded = round(some_num, 4)
    if rounded < 1:
        rounded *= 100
        unit = 'cm'
    else:
        unit = 'm'
    return str(rounded) + unit

def get_world_offset(left_line, right_line):
    best_x_left, best_y_left = left_line.get_x_and_y()
    best_x_right, best_y_right = right_line.get_x_and_y()
    width_of_lane_on_screen = best_x_right[-1] - best_x_left[-1]
#     print('best_x_left', best_x_left[-1])
#     print('best_x_right', best_x_right[-1])
#     print('width_of_lane_on_screen', width_of_lane_on_screen)
    distance_from_center = best_x_right[-1] - width_of_lane_on_screen / 2 - img_width / 2
    return distance_from_center * xm_per_pix
    
def get_lane_curvature(left_line, right_line):
    # average the curvature between the lane lines to compensate for some wobble
    return (left_line.average_curvature + right_line.average_curvature) / 2
    
def annotate_image(img, left_line, right_line):
    curvature = get_lane_curvature(left_line, right_line)
    offset = get_world_offset(left_line, right_line)
        
    if curvature < somewhat_straight_curvature_radius:
        curvature_string = format_in_meters(curvature)
    else:
        curvature_string = 'straight line-ish'
    
#     print('offset', offset)
    if abs(offset) < 0.01:
        offset_string = 'centered-ish'        
    elif offset < 0:
        offset_string = format_in_meters(abs(offset)) + ' right'
    else:
        offset_string = format_in_meters(abs(offset)) + ' left'

    cv2.putText(img, 'curvature: ' + curvature_string, (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(img, 'offset: ' + offset_string, (25, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


def process_image(img, debug=False):
#     debug = True
    img_height, img_width, num_channels = img.shape

    # step 1: undistort using some previously determined values
    undistorted = cv2.undistort(img, mtx, dist, None)
    
    # step 2: find some edges
    edges = find_edges(undistorted, debug)
#     plot_image(make_3_channel_image_from_binary_channel(edges))

    # step 3: transform image using a hardcoded trapezoid transform (outside this method)
    warped = cv2.warpPerspective(edges, M, (img_width, img_height), flags=cv2.INTER_LINEAR)
    image_of_lane = get_image_of_found_lane(warped, img_height, debug)

    unwarped = cv2.warpPerspective(image_of_lane, M_inv, (img_width, img_height), flags=cv2.INTER_LINEAR)
    weighted_final = weighted_img(unwarped, undistorted, 1, 0.5)
    annotate_image(weighted_final, left_line, right_line)
    
    if debug is True:
        plot_image(undistorted)
        plot_image(weighted_final)

    return weighted_final

# result = process_image(mpimg.imread('test_images/test1.jpg'), True)
# result = process_image(mpimg.imread('test_images/test2.jpg'), True)
# result = process_image(mpimg.imread('test_images/test3.jpg'), True)
# result = process_image(mpimg.imread('test_images/test4.jpg'), True)
# result = process_image(mpimg.imread('test_images/test5.jpg'), True)
# result = process_image(mpimg.imread('test_images/test6.jpg'), True)
# result = process_image(mpimg.imread('test_images/test7.jpg'), True)
# result = process_image(mpimg.imread('test_images/test9.jpg'), True)
# result = process_image(mpimg.imread('test_images/straight_lines1.jpg'), True)
# result = process_image(mpimg.imread('test_images/straight_lines2.jpg'), True)
# result = process_image(mpimg.imread('test_images/color-shadow-example.jpg'), True)
# plot_image(result)

def process_video(input_filename, output_filename, subclip_range=None):
    clip = VideoFileClip(input_filename)
    if subclip_range is not None:
        start_time, end_time = subclip_range
        clip = clip.subclip(start_time, end_time)
    clip_output = clip.fl_image(process_image)
    %time clip_output.write_videofile(output_filename, audio=False)
    

process_video('project_video.mp4', 'project_video_with_lines.mp4')
# process_video('challenge_video.mp4', 'challenge_video_with_lines.mp4')

# plot_images(orig_img, result)




[MoviePy] >>>> Building video project_video_with_lines.mp4
[MoviePy] Writing video project_video_with_lines.mp4



 12%|█▏        | 152/1261 [01:29<10:48,  1.71it/s]


 24%|██▍       | 303/1261 [02:57<11:29,  1.39it/s]


 36%|███▌      | 454/1261 [04:52<10:23,  1.29it/s]


 48%|████▊     | 605/1261 [06:47<05:55,  1.84it/s]


...

 72%|███████▏  | 907/1261 [10:30<04:34,  1.29it/s]


 84%|████████▍ | 1057/1261 [13:00<02:29,  1.37it/s]


 96%|█████████▌| 1205/1261 [15:08<00:46,  1.21it/s]


100%|█████████▉| 1260/1261 [15:51<00:00,  1.24it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_with_lines.mp4 

CPU times: user 27min 14s, sys: 1min 22s, total: 28min 37s
Wall time: 15min 51s
